In [1]:
%load_ext autoreload
%autoreload 2

import trio
import pandas as pd
import numpy as np
import polygon_streaming_api
import polygon_rest_api
import polygon_backfil
import bars

import pandas_bokeh
pandas_bokeh.output_notebook()
pd.set_option('plotting.backend', 'pandas_bokeh')

# from IPython.core.debugger import set_trace; set_trace(context=10)

In [2]:
tlist = polygon_backfil.get_ticks_date(symbol='GLD', date='2020-06-01', tick_type='trades')

Downloaded:  50000 GLD ticks; latest time(NYC):  2020-06-01 15:26:34.415337514-04:00
Downloaded:  10213 GLD ticks; latest time(NYC):  2020-06-01 20:00:00.001730244-04:00


In [3]:
qlist = polygon_backfil.get_ticks_date(symbol='GLD', date='2020-06-01', tick_type='quotes')

Downloaded:  50000 GLD ticks; latest time(NYC):  2020-06-01 10:01:02.319517790-04:00
Downloaded:  50000 GLD ticks; latest time(NYC):  2020-06-01 10:53:33.443750450-04:00
Downloaded:  50000 GLD ticks; latest time(NYC):  2020-06-01 11:41:03.460294462-04:00
Downloaded:  50000 GLD ticks; latest time(NYC):  2020-06-01 13:09:58.678488294-04:00
Downloaded:  50000 GLD ticks; latest time(NYC):  2020-06-01 14:58:00.074625257-04:00
Downloaded:  31424 GLD ticks; latest time(NYC):  2020-06-01 20:00:00.023348634-04:00


In [4]:
tdf = polygon_backfil.trades_to_df(tlist)
tdf

,tick_epoch,exchange_epoch,sequence,trade_id,exchange_id,price,size,tick_dt
0,1590998400018988632,1590998400018857216,28901,52983525027893,11,163.68,318,2020-06-01 08:00:00.018988632+00:00
1,1590998430367082112,1590998430366254428,29101,3472338191832597853,12,163.62,30,2020-06-01 08:00:30.367082112+00:00
2,1590998460822877103,1590998460822194403,29501,3472338191832598583,12,163.60,200,2020-06-01 08:01:00.822877103+00:00
3,1590998588275295490,1590998588274619881,29701,3472338191832602969,12,163.69,500,2020-06-01 08:03:08.275295490+00:00
4,1590998643251267867,1590998643250974976,29801,52983525027914,11,163.67,6,2020-06-01 08:04:03.251267867+00:00
...,...,...,...,...,...,...,...,...
60207,1591055987639956383,1591055987639489000,51862401,52983525830197,8,163.66,44,2020-06-01 23:59:47.639956383+00:00
60208,1591055987640044371,1591055987639489000,51862501,52983525830198,8,163.50,800,2020-06-01 23:59:47.640044371+00:00
60209,1591055987640257222,1591055987640136704,51862601,52983577070413,11,163.67,56,2020-06-01 23:59:47.640257222+00:00
60210,1591056000001694155,1591056000001318656,51865101,52983577070667,11,163.66,0,2020-06-02 00:00:00.001694155+00:00


In [5]:
qdf = polygon_backfil.quotes_to_df(qlist)
qdf

,tick_epoch,exchange_epoch,sequence,bid_exchange_id,ask_exchange_id,bid_price,ask_price,bid_size,ask_size,tick_dt
0,1590998400298769830,1590998400298179790,18801,11,12,163.55,163.69,12,10,2020-06-01 08:00:00.298769830+00:00
1,1590998400699578926,1590998400699049483,20401,11,12,163.55,163.68,12,10,2020-06-01 08:00:00.699578926+00:00
2,1590998400827058311,1590998400826950400,21001,12,12,163.55,163.68,10,10,2020-06-01 08:00:00.827058311+00:00
3,1590998400827546217,1590998400827031829,21201,11,12,163.55,163.68,2,10,2020-06-01 08:00:00.827546217+00:00
4,1590998400833380821,1590998400832805526,21301,11,12,163.55,163.67,2,10,2020-06-01 08:00:00.833380821+00:00
...,...,...,...,...,...,...,...,...,...,...
281414,1591055681399391083,1591055681398996000,3029791401,11,8,163.50,163.76,9,2,2020-06-01 23:54:41.399391083+00:00
281415,1591055704564545051,1591055704564204000,3029798501,11,8,163.50,163.90,9,1,2020-06-01 23:55:04.564545051+00:00
281416,1591055778582643169,1591055778582472192,3029822301,11,8,163.50,163.90,9,1,2020-06-01 23:56:18.582643169+00:00
281417,1591055921710835001,1591055921710427000,3029870601,11,8,163.50,163.90,9,1,2020-06-01 23:58:41.710835001+00:00


In [7]:
from os.path import join

In [6]:
%time qdf['tick_dt2'] = pd.to_datetime(qdf['tick_epoch'], utc=True, unit='ns')

CPU times: user 3.51 ms, sys: 1.17 ms, total: 4.68 ms
Wall time: 3.38 ms
